In [323]:
import pandas as pd
import json
from tqdm.autonotebook import tqdm

/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
df = pd.read_csv("data/AKG/Test Collection/AKGG/akg_standard_akgg_property_rele.csv")

In [13]:
df.property.nunique()

296

In [35]:
import json

with open("data/AKG/Formal Run Topics/AKGG_Formal_Run_Topic.json") as json_file:
    data = json.load(json_file)
    qid, query, entity, entityType, action = [], [], [], [], []
    for p in data['queries']:
        qid.append(p['queryId'])
        query.append(p['query'])   
        entity.append(p['entity'])
        entityType.append(' '.join(p['entityTypes']))    
        action.append(p['action'])
topic = pd.DataFrame({"query_id": qid, "query": query, "entity": entity, "entityType": entityType, "action":action})

In [99]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [156]:

with open("data/AKG/Participants Runs/AKGG/akgg-formalrun-cuis.json") as json_file:
    data = json.load(json_file)
    run = AutoVivification()
    for p in data['runs']:
        for res in p['results']:
            for prop in res['properties']:
                run[p['runid']][str(res['queryid'])][str(prop['property'])] = prop['rank']


In [31]:
df = df.merge(topic, how="inner", on="query_id")
df['query'] = df['query'].str.lower()
df['action'] = df['action'].str.lower()
df['entityType'] = df['entityType'].str.lower()
df['entity'] = df['entity'].str.lower()

In [37]:
df.entityType.nunique()

41

In [39]:
df.entity.nunique()

229

In [367]:
import collections
qrel = collections.defaultdict(dict)
for qid, prop, label in df[['query_id', 'property', 'rele_label']].values:
    qrel[str(qid)][str(prop)] = int(label)
    

In [368]:
selected_qids = df[df['query'].isin(list(entity2wiki.keys()))].query_id.unique()

In [449]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

res = []
for i in qrel:
    
    if int(i) not in selected_qids:
        continue
    # dict of { document ID: relevance level }
    qrels = qrel[str(i)] 
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
#     grades = list(set(qrels.values())) + [0]
    ranked_list = list(run['1'][str(i)].keys()) # a list of document IDs
#     ranked_list = naive[str(i)] # a list of document IDs
#     ranked_list =  {**naive[str(i)],  **run['1'][str(i)]}
#     for k in ranked_list:
#         ranked_list[k] *= -1

    # labeling: [doc_id] -> [(doc_id, rel_level)]
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)

    # compute the number of documents for each relevance level
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)

    # Let's compute nDCG@5
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    res.append(result)

print(np.mean(res))

0.5196837307322266


In [445]:
qrels

{'potentialAction': 4,
 'url': 4,
 'description': 3,
 'name': 2,
 'alternateName': 2}

In [446]:
labeled_ranked_list

[('name', 2),
 ('url', 4),
 ('description', 3),
 ('identifier', None),
 ('image', None),
 ('alternateName', 2),
 ('additionalType', None),
 ('potentialAction', 4)]

In [307]:
df_wiki = pd.read_csv("data/wikihowSep.csv")
df_wiki['headline'] = df_wiki['headline'].str.replace("\n", "")
df_wiki['title'] = df_wiki['title'].str.replace("How to", "")

In [787]:
df_wiki['overview'] = df_wiki['overview'].astype(str)
df_wiki['headline'] = df_wiki['headline'].astype(str)
df_wiki['text'] = df_wiki['text'].astype(str)
df_wiki['sectionLabel'] = df_wiki['sectionLabel'].astype(str)
df_wiki['title'] = df_wiki['title'].astype(str)

In [309]:
df_wiki['title'] = df_wiki['title'].str.lower()

In [340]:
%%time
queries = [q.lower() for q in df['query'].unique()]
query2wiki = collections.defaultdict(object)
for entity in tqdm(queries, total=len(queries)):
    df_ = df_wiki[df_wiki.title.str.contains(entity, na=False)]
    if len(df_) > 0:
        query2wiki[entity] = df_


CPU times: user 2min 11s, sys: 668 ms, total: 2min 12s
Wall time: 2min 12s


In [682]:
query2id = dict(zip(df['query'], df['query_id']))
query2entity = dict(zip(df['query'], df['entity']))
query2action = dict(zip(df['query'], df['action']))

In [456]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jarana/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jarana/anacon

In [470]:
all_properties = []
for i in run["1"]:
    all_properties.extend(list(run["1"][i].keys()))

In [481]:
matched_wiki = df_wiki[df_wiki.title.str.contains('|'.join(queries), na=False)]
wiki_all_text = []
for col in df_wiki.columns:
    wiki_all_text.extend(matched_wiki[col].unique())

{' ',
 ' acupuncture effects',
 'abandon',
 'abraham lincoln',
 'acapulco',
 'actionStatus',
 'activeIngredient',
 'actor',
 'acupuncture',
 'additionalProperty',
 'additionalType',
 'address',
 'adidas',
 'adverseEvent',
 'affiliation',
 'agent',
 'aggregateRating',
 'alan turing',
 'album',
 'all about my mother',
 'all about my mother plot',
 'allow',
 'alternateName',
 'alumni',
 'alumniOf',
 'amenityFeature',
 'angola',
 'announce retirement',
 'anti-gravity',
 'anti-gravity effect',
 'aol',
 'apgar score',
 'apple cider',
 'apply',
 'apply for a job',
 'apply for a job ',
 'apply for citizenship',
 'areaServed',
 'aristotle',
 'associatedPathophysiology',
 'asthma',
 'attend ceremony',
 'attendee',
 'audience',
 'audio',
 'author',
 'automotive battery',
 'availableChannel',
 'award',
 'baby shower',
 'bacardi',
 'back pain',
 'back to the future',
 'bake',
 'banana bread',
 'barbecue sauce',
 'barbie hsu',
 'batman',
 'beat niculescu',
 'become a f1 driver',
 'become man',
 'bec

In [505]:
topic_all_text = []
for col in ["property", "query", "entity", "entityType", "action"]:
    topic_all_text.extend(df[col].unique())

In [510]:
corpus = [str(i) for i in wiki_all_text + topic_all_text + all_properties]

In [511]:
MAX_NUM_WORDS = 100000
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(corpus)

In [513]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 34795 unique tokens.


In [522]:
def get_pretrain_embeddings(path, word_index, EMBEDDING_DIM=300):
    MAX_NUM_WORDS = len(word_index)
    BASE_DIR = path + 'data/'
    GLOVE_DIR = os.path.join(BASE_DIR, 'w2v')
    print('Indexing word vectors.')

    embeddings_index = {}
    with open(os.path.join(GLOVE_DIR, 'glove.42B.300d.txt'), encoding="utf-8") as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, 'f', sep=' ')
            embeddings_index[word] = coefs

    print('Found %s word vectors.' % len(embeddings_index))

    print('Preparing embedding matrix.')

    # prepare embedding matrix
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    found = 0
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if embedding_vector.shape[0] == 0:
                continue
            embedding_matrix[i] = embedding_vector
            found += 1

    print("Token num: %d, Found Tokens: %d" % (len(word_index), found))

    # load pre-trained word embeddings into an Embedding layer
    embedding_layer = Embedding(num_words,
                                EMBEDDING_DIM,
                                embeddings_initializer=Constant(embedding_matrix))

    return embedding_layer

In [523]:
from Models import getDSSM, getRanker

path = "/Users/jarana/workspace/WikiHow-Task-Based/"
max_words = len(word_index)
embedding_layer = get_pretrain_embeddings(path, word_index)

Indexing word vectors.
Found 1917494 word vectors.
Preparing embedding matrix.
Token num: 34795, Found Tokens: 29582


In [529]:
MAX_SEQUENCE_LENGTH = max([len(i.split()) for i in corpus])


In [532]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
    t_inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
#     lstm = LSTM(100)
    q_emb = GlobalMaxPooling1D()(embedding_layer(q_inp))
    t_emb = GlobalMaxPooling1D()(embedding_layer(t_inp))
#     q_emb = lstm(embedding_layer(q_inp))
#     t_emb = lstm(embedding_layer(t_inp))
    pred = Dot(-1, normalize=True)([q_emb, t_emb])
    model = Model([q_inp, t_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
model = getDSSM(embedding_layer, MAX_SEQUENCE_LENGTH)

In [ ]:
x_task = pad_sequences(tokenizer.texts_to_sequences(data["t"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query1 = pad_sequences(tokenizer.texts_to_sequences(data["q1"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
x_query2 = pad_sequences(tokenizer.texts_to_sequences(data["q2"].tolist()), maxlen=MAX_SEQUENCE_LENGTH)
y = np.array([[i,j] for i,j in zip(data.label1.values, data.label2.values)])

In [595]:
run["1"]

{'1': {'result': 1,
  'target': 2,
  'participant': 3,
  'identifier': 4,
  'location': 5,
  'instrument': 6,
  'name': 7,
  'object': 8,
  'error': 9,
  'image': 10,
  'description': 11,
  'agent': 12,
  'url': 13,
  'additionalType': 14,
  'startTime': 15,
  'potentialAction': 16,
  'endTime': 17,
  'actionStatus': 18,
  'alternateName': 19},
 '2': {'name': -1,
  'participant': -2,
  'result': -3,
  'location': -4,
  'target': -5,
  'object': -6,
  'agent': -7,
  'image': -8,
  'identifier': -9,
  'error': -10,
  'instrument': -11,
  'description': -12,
  'url': -13,
  'startTime': -14,
  'endTime': -15,
  'alternateName': -16,
  'additionalType': -17,
  'actionStatus': -18,
  'potentialAction': -19},
 '3': {'participant': 1,
  'location': 2,
  'name': 3,
  'object': 4,
  'result': 5,
  'target': 6,
  'identifier': 7,
  'instrument': 8,
  'image': 9,
  'error': 10,
  'agent': 11,
  'description': 12,
  'url': 13,
  'startTime': 14,
  'endTime': 15,
  'potentialAction': 16,
  'additio

In [ ]:
from pyNTCIREVAL import Labeler
from pyNTCIREVAL.metrics import MSnDCG, nERR, nDCG

def evaluate(qrels, ranked_list):
    res = []
    grades = [1,2,3,4] # a grade for relevance levels 1 and 2 (Note that level 0 is excluded)
    labeler = Labeler(qrels)
    labeled_ranked_list = labeler.label(ranked_list)
    rel_level_num = 5
    xrelnum = labeler.compute_per_level_doc_num(rel_level_num)
    metric = MSnDCG(xrelnum, grades, cutoff=10)
    result = metric.compute(labeled_ranked_list)
    return result

[]

In [705]:
for col in df_wiki.columns:
    res = []
    for query in query2wiki:
        qrels = qrel[str(query2id[query])]
#         cand_properties = list(run['1'][str(query2id[query])].keys())
        cand_properties = list(run['1'][str(query2id[query])].keys()) + list(qrels.keys())
        entity = str(query2entity[query])
        action = str(query2action[query])
        wikis = [str(i) for i in query2wiki[query][col].unique()]
        x_wikis = pad_sequences(tokenizer.texts_to_sequences(wikis), maxlen=MAX_SEQUENCE_LENGTH)

        rank = {}
        for prop in cand_properties:
            x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
            rank[prop] = np.max(model.predict([x_props, x_wikis]))
        rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
        res.append(evaluate(qrels, rank))
    

    print(col, np.mean(res))

overview 0.4878393658549674
headline 0.5275873391574725
text 0.4987795913292341
sectionLabel 0.5243990747826948
title 0.5345313329574838


In [650]:
len(df), df.query_id.nunique(), df.property.nunique(), df.entityType.nunique(), df['query'].nunique()

(3796, 229, 296, 41, 229)

In [676]:
trainIds, testIds = [], []
for name, group in df.groupby("entityType"):
    if group.query_id.nunique() > 1:
        ids = list(group.query_id.unique())
        mid = int(group.query_id.nunique() / 2)
        trainIds.extend(ids[:mid])
        testIds.extend(ids[mid:])
    else:
        ids = list(group.query_id.unique())
        trainIds.extend(ids)

In [677]:
df_train = df[df.query_id.isin(trainIds)]
df_test = df[df.query_id.isin(testIds)]

In [678]:
len(df_test), len(df_train)

(1857, 1939)

In [680]:
len(df)

3796

In [767]:
from keras.models import Sequential
from keras import Input, Model
from keras.layers import GlobalMaxPooling1D, Dot, LSTM, Dense, Concatenate


def getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH):

    q_inp = Input(shape=(MAX_QUERY_SEQUENCE_LENGTH,))
    d_inp = Input(shape=(MAX_DOC_SEQUENCE_LENGTH,))
    
    q_lstm = LSTM(100)
    d_lstm = LSTM(100)
    q_emb = q_lstm(embedding_layer(q_inp))
    d_emb = d_lstm(embedding_layer(d_inp))
    concat = Concatenate()([q_emb, d_emb])
    dense = Dense(PROPERTY_NUN, activation='sigmoid')
    pred = dense(concat)
    model = Model([q_inp, d_inp], pred)

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

for col in df_wiki.columns:
    PROPERTY_NUN = df.property.nunique() 
    MAX_QUERY_SEQUENCE_LENGTH = np.max([len(i.split()) for i in df.entity.unique()]) + np.max([len(i.split()) for i in df.action.unique()])
    MAX_DOC_SEQUENCE_LENGTH = np.max([len(str(i).split()) for i in df_wiki[col].unique()])
    model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH)
    
    break
#     for qid in trainIds:
#         label = qrel[qid].keys()
        
#         entity = str(query2entity[query])
#         action = str(query2action[query])
        

In [706]:
from keras.utils import to_categorical

In [721]:
prop2id = {j:i for i, j in zip(range(df.property.nunique()), df.property.unique())}
id2prop = {i:j for i, j in zip(range(df.property.nunique()), df.property.unique())}

In [735]:
PROPERTY_NUN = df.property.nunique() 


In [848]:
df_wiki.columns

Index(['overview', 'headline', 'text', 'sectionLabel', 'title'], dtype='object')

In [863]:
df_wiki.head()

,overview,headline,text,sectionLabel,title
0,So you're a new or aspiring artist and your c...,Sell yourself first.,"Before doing anything else, stop and sum up y...",Steps,sell fine art online
1,"If you want to be well-read, then, in the wor...",Read the classics before 1600.,Reading the classics is the very first thing ...,Reading the Classics,be well read
2,So you're a new or aspiring artist and your c...,Join online artist communities.,Depending on what scale you intend to sell yo...,Steps,sell fine art online
3,So you're a new or aspiring artist and your c...,Make yourself public.,Get yourself out there as best as you can by ...,Steps,sell fine art online
4,So you're a new or aspiring artist and your c...,Blog about your artwork.,"Given the hundreds of free blogging websites,...",Steps,sell fine art online


In [867]:
# for col in ['overview', 'headline', 'text', 'sectionLabel', 'title']:
for col in ['title']:
    res = []
    train_id, train_query, train_doc, train_y = [], [], [], []
    test_id, test_query, test_doc, test_y = [], [], [], []
    
    for key in query2wiki:
        qid = query2id[key]
        qrels = qrel[str(qid)]
#       convert prop to prop_id
        label = to_categorical([prop2id[i] for i in list(qrels.keys())], num_classes=PROPERTY_NUN)
        label = np.sum(label, axis=0)
        
        entity = str(query2entity[key])
        action = str(query2action[key])
        
        for wiki in query2wiki[key][col].unique():
            query = pad_sequences(tokenizer.texts_to_sequences([entity + " " + action] * len(wikis)), maxlen=MAX_QUERY_SEQUENCE_LENGTH)[0]
            
            doc = pad_sequences(tokenizer.texts_to_sequences(str(wiki)), maxlen=MAX_DOC_SEQUENCE_LENGTH)[0]
            
            train_query.append(query) if qid in trainIds else test_query.append(query)
            train_doc.append(doc) if qid in trainIds else test_doc.append(doc)
            train_y.append(label) if qid in trainIds else test_y.append(label)
            train_id.append(qid) if qid in trainIds else test_id.append(qid)
#         rank = {}
#         for prop in cand_properties:
#             x_props = pad_sequences(tokenizer.texts_to_sequences([entity+ " " + action + " " + prop] * len(x_wikis)), maxlen=MAX_SEQUENCE_LENGTH)
#             rank[prop] = np.max(model.predict([x_props, x_wikis]))
#         rank = [i[0] for i in sorted(rank.items(), key=lambda x: x[1], reverse=True)]
#         res.append(evaluate(qrels, rank))
    train_query = np.array(train_query)
    train_doc = np.array(train_doc)
    train_y = np.array(train_y)
    test_query = np.array(test_query)
    test_doc = np.array(test_doc)
    test_y = np.array(test_y)
    break


In [868]:
model = getPropModel(embedding_layer, PROPERTY_NUN, MAX_QUERY_SEQUENCE_LENGTH, MAX_DOC_SEQUENCE_LENGTH)


In [ ]:
model.fit([train_query, train_doc], train_y)

Epoch 1/1
 320/1517 [=====>........................] - ETA: 1:30 - loss: 0.6233 - acc: 0.7673

In [812]:
model.evaluate([test_query, test_doc], test_y)

1074/1074 [==============================] - 12s 11ms/step


[0.15142626305643184, 0.9417276466802956]

In [843]:
for col in df_wiki.columns:
    res = []
    for query in query2wiki:
        qid = query2id[query]
        if qid not in test_id:
            continue
        qrels = qrel[str(query2id[query])]
        cand_properties = list(run['1'][str(query2id[query])].keys())
        res.append(evaluate(qrels, cand_properties))
    print(col, np.mean(res))    
    break
    



overview 0.5134289962516724


In [851]:
pred = model.predict([test_query, test_doc])

In [859]:
pred = np.argsort(pred)[:,::-1][:,:20]

In [860]:
runLSTM = {}
for i, j in zip(test_id, pred):
    runLSTM[i] = [id2prop[k] for k in j]


In [862]:
for col in df_wiki.columns:
    res = []
    for query in query2wiki:
        qid = query2id[query]
        if qid not in test_id:
            continue
        qrels = qrel[str(query2id[query])]
        cand_properties = runLSTM[qid]
        print(query, cand_properties)
        res.append(evaluate(qrels, cand_properties))
    print(col, np.mean(res))    
    break
    



hit and run ['identifier', 'description', 'image', 'name', 'review', 'potentialAction', 'additionalType', 'url', 'brand', 'logo', 'alternateName', 'medicineSystem', 'recognizingAuthority', 'study', 'isRelatedTo', 'model', 'comprisedOf', 'legalStatus', 'relatedCondition', 'offers']
mediation ['identifier', 'image', 'description', 'name', 'review', 'potentialAction', 'additionalType', 'url', 'logo', 'brand', 'medicineSystem', 'alternateName', 'recognizingAuthority', 'study', 'isRelatedTo', 'model', 'legalStatus', 'offers', 'comprisedOf', 'relatedTherapy']
pull up ['identifier', 'description', 'image', 'name', 'review', 'potentialAction', 'additionalType', 'url', 'brand', 'logo', 'alternateName', 'medicineSystem', 'recognizingAuthority', 'study', 'isRelatedTo', 'offers', 'legalStatus', 'relatedCondition', 'comprisedOf', 'model']
run fast ['identifier', 'description', 'image', 'name', 'review', 'potentialAction', 'additionalType', 'url', 'brand', 'logo', 'medicineSystem', 'alternateName', 

In [858]:
np.argsort(pred)[:,::-1][:,:20]

array([[ 18,   8,  14, ..., 146, 166,  61],
       [ 18,   8,  14, ..., 146, 166,  61],
       [ 18,  14,   8, ...,  61, 164, 167],
       ...,
       [ 18,   8,  14, ..., 146,  61, 167],
       [ 18,   8,  14, ..., 146,  61, 167],
       [ 18,   8,  14, ..., 146,  61, 167]])